In [5]:
%run ./Page.ipynb
%run ./RegisterDevicePage.ipynb
%run ./ContractsPage.ipynb
%run ./GridComponent.ipynb

In [3]:
import time
import json
from selenium import webdriver
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC 

class TicTatToeApp:
    def __init__(self):
        self.wd = webdriver.Chrome()
        self.wd.get('https://jerichokain.github.io/TicTacVue')
        #self.wd.get('http://127.0.0.1:8081/')
        time.sleep(2) #gets cache and dom renders
        self.pages = dict([(page.route, page) for page in Page.__subclasses__()])
    
    def __del__(self):
        self.wd.close()
    
#     def _get_shadow_root(self, elt):
#         return self.wd.execute_script('return arguments[0].shadowRoot', elt)
    
    def get_title(self):
        elt = self.wd.find_element_by_tag_name('my-app')
        return self._get_shadow_root(elt).find_element_by_id('main-title').text
    
    def get_page(self, test):
        elt = self.wd.find_element_by_tag_name('my-app')
        return self.pages[elt.get_attribute('page')](self, test)

    def _get_component(self, page_component):
        elt = self.wd.find_element_by_tag_name('my-app')
        elt2 = self._get_shadow_root(elt)
        elt3 = elt2.find_element_by_css_selector(page_component)
        return self._get_shadow_root(elt3)
                      
    def _click_button(self, web_element, text):
        buttons = web_element.find_elements_by_css_selector('paper-button')
        for button in buttons:
            if button.text == text:
                button.click()
                time.sleep(0.5)
                return
        raise AssertionError(f'No paper-button with text="{text}" found. Available buttons are [{", ".join([b.text for b in buttons])}].')
    
    def _css_has_text(self, web_element, css, text):
        elts = filter(lambda e: 'display: none' not in [s.strip(' ') for s in e.get_attribute('style').split(';')], 
                      web_element.find_elements_by_css_selector(css))
        texts = [e.text for e in elts]
        if text in texts:
            return True
        raise AssertionError(f'No {css} with text="{text}" found. Available matches are [{", ".join(texts)}].')
    
    def _get_input_by_label(self, web_element, label):
        elts = list(filter(lambda e: 'display: none' not in [s.strip(' ') for s in e.get_attribute('style').split(';')],
                      web_element.find_elements_by_css_selector('paper-input')))
        for elt in elts:
            if elt.get_attribute('label') == label:
                return elt
        raise AssertionError(f'No paper-input with label="{label}" found. Available inputs are [{", ".join([e.get_attribute("label") for e in elts])}].')
    
#mma = MaterialsMobileApp()